Линейная регрессия (Linear regression) — один из простейший алгоритмов машинного обучения, описывающий зависимость целевой переменной от признака в виде линейной функции y = kx + b. В данном случае была представлена простая или парная линейная регрессия, а уравнение вида f_{w, b}(x) = w_0x_0 + w_1x_1 +... + w_{n}x_{n} + b = w \cdot x + b называется множественной линейной регрессией, где b — смещение модели, w — вектор её весов, а x — вектор признаков одного обучающего образца.

К другим условиям определения линейной регрессии относятся гомоскедастичность (дисперсия остатков постоянна и конечна), а также отсутствие мильтиколлинеарности (линейной зависимости между признаками).

In [84]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale, PolynomialFeatures
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score

### Аналитическое решение в виде w = (X_T * X)**-1 * X_T * y

Где w - веса(коэфициенты) признаков x, w0 - смещение(свободный член) который сдвигает нашу линию по y, иначе у нас бы всегда линия начиналась бы с нуля

In [85]:
class MatrixLinearRegression:

    def fit(self, X, y):
        X = np.insert(X, 0, 1, axis=1) #Добавляем столбец с 1-ми в нашу матрицу чтобы учесть bias(w0 при y = w0 + w1*x.....)
        XT_X_inv = np.linalg.inv(X.T @ X) #находим (X * X_T)**-1 чтобы подстваить в w = (X * X_T)**-1 * X_T * y
        weights = np.linalg.multi_dot([XT_X_inv, X.T, y])  #находим сами веса w через матричное умножение
        self.bias, self.weights = weights[0], weights[1:] #Присваиваем нулевой столбец в bias(w0), а остальные в веса признаков 

    def predict(self, X_test):
        return X_test @ self.weights + self.bias #Возвращаем y = X * w + w0

Как работает наше аналитическое решение под капотом?
Наш метод тупо берёт всю матрицу данных и вычисляет единственное оптимальное решение с помощью формулы нормального уравнения:
1)Без градиентного спуска, без обновления весов, без итераций.
2)Без функций потерь, потому что формула сразу выдаёт оптимальные веса, минимизирующие среднеквадратичную ошибку (MSE).
3) Это просто линейная алгебра: перемножение матриц, обращение, и вуаля — у нас есть веса!

Почему это усреднённое решение?
Линейная регрессия ищет прямую, которая максимально приближает данные, минимизируя среднеквадратичное отклонение (MSE).

То есть:
Берём все точки (матрицу X)
Считаем общее направление их зависимости от y (перемножаем, инвертируем).
Выводим единственный набор коэффициентов w, который даёт оптимальную прямую.
Это похоже на усреднение данных в пространстве n-мерных признаков.

Недостатки этого метода:
1) Высокая вычислительная сложность
2) Плохо работает с мультиколлинеарностью:
    Если признаки сильно коррелируют, X может стать необратимой (сингулярной) матрицей.
    В таком случае матрица не инвертируется, и код сломается.
    Решение: использовать регуляризацию (L2-регрессия, Ridge).

### Решение при помощи Градиентного спуска

Градиентный спуск — это итеративный метод оптимизации, 
который помогает найти минимум функции потерь путем обновления весов модели в направлении отрицательного градиента.

Где learning_rate - шаги которые дают нам двиагться вперед а точнее умножая эти шаги мы ищем идеальные веса.
learning_rate определяет, насколько сильно мы корректируем наши веса на каждом шаге.
Если шаг слишком большой, мы можем "перепрыгнуть" минимум и не сойтись. Если шаг слишком маленький, алгоритм будет сходиться очень медленно.
То есть, каждый раз мы делаем шаг в сторону минимума, но не "прыгаем" слишком далеко, чтобы не пропустить его.

In [86]:
class GDLinearRegression:
    def __init__(self, learning_rate=0.01, tolerance=1e-8): #устанавливаем нужные значения learning_rate
        self.learning_rate = learning_rate
        self.tolerance = tolerance

    def fit(self, X, y):
        n_samples, n_features = X.shape #n_samples = len of rows, n_features = num of columns
        self.bias, self.weights = 0, np.zeros(n_features) # устанавливаем начальные веса
        previous_db, previous_dw = 0, np.zeros(n_features) # устанавливаем начальные градиенты
        
        while True:
            y_pred = X @ self.weights + self.bias #Делаем прогноз с начальными значениями
            db = 1 / n_samples * np.sum(y_pred - y) #Ищем градиент по bias-у
            dw = 1 / n_samples * X.T @ (y_pred - y) #Ищем градиаент по весу w

            self.weights -= self.learning_rate * dw #Обновляем веса умножая на наш "шаг"
            self.bias -= self.learning_rate * db # Обновляем bias умножая на наш "шаг"

            #Находим разницу прошлого и нынешнего градиента чтобы при случае минимальног различие в ту или иную сторону остановить обучение ведь достигается минимум
            diff_db = np.abs(previous_db - db) 
            diff_dw = np.abs(previous_dw - dw)

            #Кусок кода который остановит обучения при приюлижения к минимуму 
            if diff_db < self.tolerance:
                if diff_dw.all() < self.tolerance:
                    break
            previous_db = db
            previous_dw = dw       

    def predict(self, X_test):
        return X_test @ self.weights + self.bias


Вообще как работает градиентный спуск здесь:
Мое обьяснение: Здесь изначально bias и веса у нас заданы как ноль, после чего мы делаем сравнение(прогноз), на основе этого прогноза пересчитывается 
наши веса и тд и тп шагая маленькиими шагами а именно как мы задали в learning_rate, а каким образом? 
Каждый шаг мы подставляем значение в наш градиент(направление нашего минимума) и обновляем наши веса и обновляем до того момента пока наша 
разница не будет такой что меньше заданного tolerance что будет значить мы подобрались к минимуму и это наш ответ.

Помощь гпт:
Градиентный спуск в этом коде работает следующим образом: сначала мы инициализируем все веса и bias значением 0, 
а затем итеративно обновляем их, минимизируя ошибку предсказания.
В каждой итерации мы сначала вычисляем прогнозируемые значения y_pred с текущими весами, 
затем находим разницу между предсказанными и реальными значениями y, которая используется для вычисления градиентов db и dw. 
Эти градиенты показывают, в каком направлении и насколько сильно нужно скорректировать bias и веса, чтобы уменьшить ошибку.

Обновление параметров происходит с использованием learning rate — небольшого шага, 
который регулирует, насколько сильно изменяются bias и веса в сторону минимума.
После каждого обновления проверяется, насколько сильно изменились db и dw по сравнению с предыдущими значениями. 
Если эти изменения становятся меньше заданного tolerance, алгоритм останавливается, так как дальнейшее обновление не приведёт к значительному улучшению.

Таким образом, градиентный спуск постепенно подбирает такие значения bias и весов,
при которых разница между предсказанными и реальными значениями минимальна, 
и алгоритм сходится к оптимальному решению.

1. Возможные проблемы и ошибки в градиентном спуске
1.1. Неправильный learning rate (шаг градиентного спуска)
Слишком маленький learning rate

Обучение слишком долгое (модель делает крошечные шаги).
Можно застрять в локальном минимуме или не достичь глобального минимума.
Решение: увеличить learning rate, но аккуратно.
Слишком большой learning rate

Модель может "перепрыгивать" минимум и не сойтись.
Ошибка может скакать хаотично.
Возможна расходимость — веса становятся огромными, и ошибка увеличивается бесконечно.
Решение: уменьшить learning rate или использовать затухающую скорость обучения (learning rate decay).
1.2. Неправильное направление градиента
Градиент указывает неверное направление, если:
Ошибка в вычислении производных.
В данных есть аномалии (выбросы), сильно влияющие на градиент.
Используется неподходящая функция потерь.
Пример ошибки:
Если использовать градиентный спуск без нормализации данных, градиенты могут быть слишком разными (один вес обновляется быстро, другой медленно), что приводит к медленной или нестабильной сходимости.

Решение:

Нормализовать данные (привести признаки к единому масштабу, например, StandardScaler).
Проверить правильность формул градиентов.
1.3. Локальные минимумы и седловые точки
Если функция потерь имеет несколько минимумов, градиентный спуск может застрять в локальном минимуме.
Если градиент почти нулевой во всех направлениях, можно попасть в седловую точку (точка, где градиент нулевой, но это не минимум).
Решение:

Попробовать другой начальный набор весов (инициализация весов случайными значениями).
Использовать разные методы градиентного спуска, например, Momentum, Adam, RMSprop, которые помогают избежать таких проблем.
1.4. Веса модели уходят в NaN или бесконечность
Такое бывает, если:
Слишком большой learning rate — градиенты становятся огромными.
Деление на 0 в вычислениях (например, из-за логарифма).
В данных есть NaN или бесконечные значения.
Решение:
Проверить данные перед обучением (np.isnan().sum(), np.isinf().sum()).
Уменьшить learning rate.
Использовать градиентный спуск с моментом (Momentum).

### Обучение моделей

In [87]:
df = pd.read_csv('multiple_linear_regression_dataset.csv')
X, y = df.iloc[:, :-1].values, df.iloc[:, -1].values
X_scaled = scale(X1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X_train_s, X_test_s, y_train, y_test = train_test_split(X_scaled, y, random_state=0)
print(df)

correlation_matrix = df.corr()
correlation_matrix.style.background_gradient(cmap='coolwarm')

    age  experience  income
0    25           1   30450
1    30           3   35670
2    47           2   31580
3    32           5   40130
4    43          10   47830
5    51           7   41630
6    28           5   41340
7    33           4   37650
8    37           5   40250
9    39           8   45150
10   29           1   27840
11   47           9   46110
12   54           5   36720
13   51           4   34800
14   44          12   51300
15   41           6   38900
16   58          17   63600
17   23           1   30870
18   44           9   44190
19   37          10   48700


,age,experience,income
age,1.000000,0.615165,0.532204
experience,0.615165,1.000000,0.984227
income,0.532204,0.984227,1.000000


#### Matrix Linear Regression

In [88]:
inear_regression = MatrixLinearRegression()
linear_regression.fit(X_train_s, y_train)
pred_res = matrix_linear_regression.predict(X_test_s)
r2 = r2_score(y_test, matrix_lr_pred_res)
mape = mean_absolute_percentage_error(y_test, matrix_lr_pred_res)

print(f'Matrix Linear regression  R2 score: {r2}')
print(f'Matrix Linear regression MAPE: {mape}', '\n')

print(f'weights: {linear_regression.bias, *linear_regression.weights}')
print(f'prediction: {pred_res}')

Matrix Linear regression  R2 score: 0.9307237996010834
Matrix Linear regression MAPE: 0.04666577176525873 

weights: (40922.38666080791, -1049.7866043333488, 8718.764356365164)
prediction: [46528.00800666 35018.47848628 49448.73803373 38604.36954966
 30788.13913983]


#### GD Linear Regression

In [89]:
linear_regression = GDLinearRegression()
linear_regression.fit(X_train_s, y_train)
pred_res = linear_regression.predict(X_test_s)
r2 = r2_score(y_test, pred_res)
mape = mean_absolute_percentage_error(y_test, pred_res)

print(f'Linear regression R2 score: {r2}')
print(f'Linear regression MAPE: {mape}', '\n')

print(f'weights: {linear_regression.bias, *linear_regression.weights}')
print(f'prediction: {pred_res}')

Linear regression R2 score: 0.9307237996011029
Linear regression MAPE: 0.04666577176525262 

weights: (40922.38666080791, -1049.7866043333488, 8718.764356365164)
prediction: [46528.00800666 35018.47848628 49448.73803373 38604.36954966
 30788.13913983]
